### 한국수출입은행 - 원화 환율(기간 지정) 및 DB 저장

In [13]:
date_from = "2023-07-03"
date_to = "2023-07-04"
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON'
api_key = 'OczbdUJk31QuSPZNx3llq7VFOEmsMuew'
RESULT_SUCCESS = 1

In [14]:
import requests
import pandas as pd

date_range = pd.date_range(start=date_from, end=date_to, freq="D")

df_fx_all = pd.DataFrame()
for dt in date_range:
    date_str = dt.strftime("%Y%m%d")
    params = {
        'authkey': api_key,
        'searchdate': date_str,
        'data': 'AP01'             # 환율
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:     # URL GET '200 정상'
        print("URL GET Error")
        pass
    df_fx = pd.json_normalize(response.json())
    if not df_fx.empty:
        if RESULT_SUCCESS not in df_fx['result'].unique():         # 성공이 아니라면
            print('Result is not successful')
            pass
        df_fx.drop('result', axis=1, inplace=True)
        df_fx.insert(0, 'date', date_str)
        df_fx_all = pd.concat([df_fx_all, df_fx], ignore_index=True)

+ RESULT  Integer 조회결과
  - 1 : 성공, 2 : DATA코드 오류, 3 : 인증코드 오류, 4 : 일일제한횟수 마감
+ CUR_UNIT	String	통화코드
+ CUR_NM	String	국가/통화명
+ TTB String 전신환(송금) 받으실때
+ TTS String 전신환(송금) 보내실때
+ DEAL_BAS_R  String 매매 기준율
+ BKPR  String	장부가격	
+ YY_EFEE_R String	년환가료율	
+ TEN_DD_EFEE_R String	10일환가료율	
+ KFTC_DEAL_BAS_R String	서울외국환중개 매매기준율	
+ KFTC_BKPR String	서울외국환중개 장부가격

In [15]:
df_fx_all.shape

(46, 11)

In [16]:
# df_fx_all.groupby('date').agg(cur = ('cur_unit', 'count'))

In [17]:
# currency = 'USD'
# df_fx_all.query('cur_unit == @currency')

In [18]:
df_fx_all.to_excel('korea_eximbank_fx.xlsx')

#### DB에 저장

In [19]:
df_fx_all_save = df_fx_all[['date', 'cur_unit', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r']]

* 데이터 전처리

In [20]:
# 숫자의 소수점(,) 없애기
df_fx_all_save.iloc[:,2:8] = df_fx_all_save.iloc[:,2:8].apply(lambda x: x.str.replace(',', ''), axis = 1)
# 통화코드에서 세 자리 초과하는 내용은 지우고 통화코드 생성: (예) JPY(100) -> JPY로 변경
df_fx_all_save.insert(2, 'cur_code', df_fx_all_save['cur_unit'].str[0:3])
df_fx_all_save = df_fx_all_save.drop('cur_unit', axis=1)

In [21]:
df_fx_all_save

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
0,20230703,AED,355.62,362.81,359.22,359,359,359.22
1,20230703,AUD,870.13,887.7,878.92,878,878,878.92
2,20230703,BHD,3464.83,3534.82,3499.83,3499,3499,3499.83
3,20230703,BND,965.8,985.31,975.56,975,975,975.56
4,20230703,CAD,985.85,1005.76,995.81,995,995,995.81
5,20230703,CHF,1459.28,1488.77,1474.03,1474,1474,1474.03
6,20230703,CNH,179.95,183.58,181.77,181,181,181.77
7,20230703,DKK,191.37,195.24,193.31,193,193,193.31
8,20230703,EUR,1425.07,1453.86,1439.47,1439,1439,1439.47
9,20230703,GBP,1658.61,1692.12,1675.37,1675,1675,1675.37


* DB 테이블에 저장

In [22]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_fx_all_save.itertuples():
    sql = "insert into fx_eximbank (date, cur_code, ttb, tts, deal_bas_r, bkpr, kftc_bkpr, kftc_deal_bas_r) \
                           values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))
conn.commit()
conn.close()